In [1]:
%matplotlib inline

import pandas as pd
import nivapy3 as nivapy
import matplotlib.pyplot as plt
import numpy as np

plt.style.use('ggplot')

# Patching missing data in Martini river inputs

[Notebook 1](https://nbviewer.jupyter.org/github/JamesSample/martini/blob/master/notebooks/01_data_pre-processing.ipynb) and [notebook 2](https://nbviewer.jupyter.org/github/JamesSample/martini/blob/master/notebooks/02_riverine_inputs.ipynb) generated daily time series of river concentrations based on available monitoring data. Results are summaries in the Excel file [here](https://github.com/JamesSample/martini/blob/master/data/tidy/martini_daily_conc_q_summary.xlsx).

There are significant gaps in the dataset for some sites and variables. This notebook attempts to patch missing data using national aggregated values, where appropriate.

In [2]:
# Read station metadata
xl_path = r'../data/tidy/martini_daily_conc_q_summary.xlsx'
stn_df = pd.read_excel(xl_path, sheet_name='stations')
stn_df.head()

,station_id,andre_id,station_name,country,missing_vars,status,comment
0,NaN,dmca003,Vigsø bukt,Denmark,All,No data available,NaN
1,oda_4000005,dmca004,West side of Nord-Jylland - Liver Å,Denmark,"TOC, DOC, SiO2",NaN,NaN
2,oda_3000002,dmca005,Hirtshals-Skagen - Uggerby Å,Denmark,"TOC, DOC, SiO2",NaN,NaN
3,NaN,dmca006,Skagen- Lyngsaa strand,Denmark,All,No data available,NaN
4,NaN,dmca007,East side of Nord-Jylland,Denmark,All,No data available,NaN


In [ ]:
# Read conc. data
xl_path = r'../data/tidy/martini_daily_conc_q_summary.xlsx'
df = pd.read_excel(xl_path, sheet_name='daily_q_concs')
df.head()

In [ ]:
# Merge SRP and TIP columns to TIP
# Denmark reports SRP, others report TIP. We're assuming these are the same
df['TIP_µg/l'] = df['TIP_µg/l'].where(df['TIP_µg/l'].notnull(), df['SRP_µg/l'])
del df['SRP_µg/l']

In [ ]:
# Melt to long format for later
df_long = df.melt(id_vars=['station_id', 'sample_date'])
df_long.head()

In [ ]:
# Dict for storing national median time series
patch_dict = {}

## 1. Denmark

All Danish stations are missing TOC, DOC and SiO2. There are 9 stations in Denmark with complete datasets for the other variables.

In [ ]:
# Danish stations with data
den_stn_df = stn_df.query('country == "Denmark"').dropna(subset=['station_id'])
den_stn_df

In [ ]:
# Get chem for Danish stations
stn_list = list(den_stn_df['station_id'].unique())
den_df = df.query('station_id in @stn_list').copy()

# Drop flow, TOC, DOC and SiO2
den_df.drop(['flow_m3/s', 'DOC_mg/l', 'TOC_mg/l', 'SIO2_µg/l'], 
            inplace=True, 
            axis='columns')

den_df = den_df.melt(id_vars=['station_id', 'sample_date'])

den_df.head()

In [ ]:
# Plot range of values for each variable
fig, axes = plt.subplots(nrows=len(den_df['variable'].unique()), ncols=1, figsize=(12,40))

# Loop over Danish data
for idx, var_name in enumerate(den_df['variable'].unique()):
    # Get data
    var_df = den_df.query('variable == @var_name')
    
    # Loop over stations
    df_list = []
    for stn_id in var_df['station_id'].unique():
        # Get data for stn
        stn_var_df = var_df.query('station_id == @stn_id')
        stn_var_df = stn_var_df[['sample_date', 'value']]
        stn_var_df.set_index('sample_date', inplace=True)
        df_list.append(stn_var_df)
        stn_var_df.plot(ax=axes[idx], legend=False)
    
    # Get median over all stations for this variable
    agg_df = pd.concat(df_list, axis='columns')
    med_df = agg_df.median(axis='columns')
    patch_dict[('Denmark', var_name)] = med_df
    
    # Plot
    med_df.plot(ax=axes[idx], legend=False, lw=3, c='k')
    axes[idx].set_title(var_name)
    axes[idx].set_xlabel('')
    
plt.tight_layout()

In [ ]:
# For consitency, adjust values for TON, N-TOT, TOP and P-TOT so that totals make sense
patch_dict[('Denmark', 'TON_µg/l')] = (patch_dict[('Denmark', 'DON_µg/l')] + 
                                       patch_dict[('Denmark', 'PON_µg/l')]) 

patch_dict[('Denmark', 'N-TOT_µg/l')] = (patch_dict[('Denmark', 'TON_µg/l')] + 
                                         patch_dict[('Denmark', 'N-NH4_µg/l')] + 
                                         patch_dict[('Denmark', 'N-SNOX_µg/l')])

patch_dict[('Denmark', 'TOP_µg/l')] = (patch_dict[('Denmark', 'DOP_µg/l')] + 
                                       patch_dict[('Denmark', 'POP_µg/l')])

patch_dict[('Denmark', 'P-TOT_µg/l')] = (patch_dict[('Denmark', 'TOP_µg/l')] + 
                                         patch_dict[('Denmark', 'TIP_µg/l')])

## 2. Norway

In [ ]:
# Norway stations with complete data
nor_stn_df = stn_df.query('country == "Norway"').query('status == "OK"')
nor_stn_df

In [ ]:
# Get chem for Norwegian stations
stn_list = list(nor_stn_df['station_id'].unique())
nor_df = df.query('station_id in @stn_list').copy()

# Drop flow
nor_df.drop(['flow_m3/s'], 
            inplace=True, 
            axis='columns')

nor_df = nor_df.melt(id_vars=['station_id', 'sample_date'])

nor_df.head()

In [ ]:
# Plot range of values for each variable
fig, axes = plt.subplots(nrows=len(nor_df['variable'].unique()), ncols=1, figsize=(12,40))

# Loop over Danish data
for idx, var_name in enumerate(nor_df['variable'].unique()):
    # Get data
    var_df = nor_df.query('variable == @var_name')
    
    # Loop over stations
    df_list = []
    for stn_id in var_df['station_id'].unique():
        # Get data for stn
        stn_var_df = var_df.query('station_id == @stn_id')
        stn_var_df = stn_var_df[['sample_date', 'value']]
        stn_var_df.set_index('sample_date', inplace=True)
        df_list.append(stn_var_df)
        stn_var_df.plot(ax=axes[idx], legend=False)
    
    # Get median over all stations for this variable
    agg_df = pd.concat(df_list, axis='columns')
    med_df = agg_df.median(axis='columns')
    patch_dict[('Norway', var_name)] = med_df
    
    # Plot
    med_df.plot(ax=axes[idx], legend=False, lw=3, c='k')
    axes[idx].set_title(var_name)
    axes[idx].set_xlabel('')
    
plt.tight_layout()

In [ ]:
# For consitency, adjust values for TON, N-TOT, TOP and P-TOT so that totals make sense
patch_dict[('Norway', 'TON_µg/l')] = (patch_dict[('Norway', 'DON_µg/l')] + 
                                       patch_dict[('Norway', 'PON_µg/l')]) 

patch_dict[('Norway', 'N-TOT_µg/l')] = (patch_dict[('Norway', 'TON_µg/l')] + 
                                         patch_dict[('Norway', 'N-NH4_µg/l')] + 
                                         patch_dict[('Norway', 'N-SNOX_µg/l')])

patch_dict[('Norway', 'TOP_µg/l')] = (patch_dict[('Norway', 'DOP_µg/l')] + 
                                       patch_dict[('Norway', 'POP_µg/l')])

patch_dict[('Norway', 'P-TOT_µg/l')] = (patch_dict[('Norway', 'TOP_µg/l')] + 
                                         patch_dict[('Norway', 'TIP_µg/l')])

## 3. Sweden

In [ ]:
# Sweden stations with near-complete data
swe_stn_df = stn_df.query('country == "Sweden"').query('status == "OK"')
swe_stn_df

In [ ]:
# Get chem for Swedish stations
stn_list = list(swe_stn_df['station_id'].unique())
swe_df = df.query('station_id in @stn_list').copy()

# Drop flow and STS
swe_df.drop(['flow_m3/s', 'STS_mg/l'], 
            inplace=True, 
            axis='columns')

swe_df = swe_df.melt(id_vars=['station_id', 'sample_date'])

swe_df.head()

In [ ]:
# Plot range of values for each variable
fig, axes = plt.subplots(nrows=len(swe_df['variable'].unique()), ncols=1, figsize=(12,40))

# Loop over Danish data
for idx, var_name in enumerate(swe_df['variable'].unique()):
    # Get data
    var_df = swe_df.query('variable == @var_name')
    
    # Loop over stations
    df_list = []
    for stn_id in var_df['station_id'].unique():
        # Get data for stn
        stn_var_df = var_df.query('station_id == @stn_id')
        stn_var_df = stn_var_df[['sample_date', 'value']]
        stn_var_df.set_index('sample_date', inplace=True)
        df_list.append(stn_var_df)
        stn_var_df.plot(ax=axes[idx], legend=False)
    
    # Get median over all stations for this variable
    agg_df = pd.concat(df_list, axis='columns')
    med_df = agg_df.median(axis='columns')
    patch_dict[('Sweden', var_name)] = med_df
    
    # Plot
    med_df.plot(ax=axes[idx], legend=False, lw=3, c='k')
    axes[idx].set_title(var_name)
    axes[idx].set_xlabel('')
    
plt.tight_layout()

In [ ]:
# For consitency, adjust values for TON, N-TOT, TOP and P-TOT so that totals make sense
patch_dict[('Sweden', 'TON_µg/l')] = (patch_dict[('Sweden', 'DON_µg/l')] + 
                                       patch_dict[('Sweden', 'PON_µg/l')]) 

patch_dict[('Sweden', 'N-TOT_µg/l')] = (patch_dict[('Sweden', 'TON_µg/l')] + 
                                         patch_dict[('Sweden', 'N-NH4_µg/l')] + 
                                         patch_dict[('Sweden', 'N-SNOX_µg/l')])

patch_dict[('Sweden', 'TOP_µg/l')] = (patch_dict[('Sweden', 'DOP_µg/l')] + 
                                       patch_dict[('Sweden', 'POP_µg/l')])

patch_dict[('Sweden', 'P-TOT_µg/l')] = (patch_dict[('Sweden', 'TOP_µg/l')] + 
                                         patch_dict[('Sweden', 'TIP_µg/l')])

## 4. Patching

In [ ]:
# Default values as medians from entire dataset where pathing at national level not possible
def_val_dict = {('Sweden', 'STS_mg/l')  : df['STS_mg/l'].median().round(0),
                ('Denmark', 'TOC_mg/l') : df['TOC_mg/l'].median().round(0),
                ('Denmark', 'DOC_mg/l') : df['DOC_mg/l'].median().round(0),
                ('Denmark', 'SIO2_µg/l'): df['SIO2_µg/l'].median().round(0),
               }

In [ ]:
# Get list of Martini outflow locations
mar_stns = stn_df.dropna(subset=['andre_id']).drop_duplicates(subset=['andre_id'])
mar_stns

In [ ]:
df_list = []

# All vars of interest, excluding flow
all_vars = [i for i in df_long['variable'].unique() if i != 'flow_m3/s']

# Loop over stations of interest
for idx, row in mar_stns.iterrows():
    mar_id = row['andre_id']
    stn_id = row['station_id']
    country = row['country']
    
    # Loop over vars
    for var_name in all_vars:
        # Do we have a complete datset based on measure data?
        stn_var_df = df_long.query('(station_id == @stn_id) and (variable == @var_name)').copy()
        
        if len(stn_var_df) == 0:
            # No data at all for this station
            stn_var_df = pd.DataFrame({'station_id': mar_id,
                                       'sample_date':pd.date_range('2015-01-01', '2017-12-31', freq='D'),
                                       'variable':var_name,
                                       'value':np.nan,
                                      })
        assert len(stn_var_df) == 1096
        
        if pd.isna(stn_var_df['value']).sum() == 0:
            # We have a complete data series
            # Change ID to mar_id
            stn_var_df['station_id'] = mar_id
            df_list.append(stn_var_df)
        else:
            # Do we have a series based on national median values?
            if (country, var_name) in patch_dict.keys():
                patch_df = patch_dict[(country, var_name)].reset_index()
                patch_df.columns = ['sample_date', 'value']
                patch_df['variable'] = var_name
                patch_df['station_id'] = mar_id
                df_list.append(patch_df)
            else:
                # Use a constant default value
                stn_var_df['value'] = def_val_dict[(country, var_name)]
                
                # Change ID to mar_id
                stn_var_df['station_id'] = mar_id
                df_list.append(stn_var_df)

# Combine and reshape
patched_df = pd.concat(df_list, axis='rows', sort=True)
patched_df.set_index(['station_id', 'sample_date', 'variable'], inplace=True)
patched_df = patched_df.unstack('variable')
patched_df.reset_index(inplace=True)
patched_df.columns = (list(patched_df.columns.get_level_values(0)[:2]) + 
                      list(patched_df.columns.get_level_values(1)[2:]))

# Reorder cols
cols = list(df.columns)
cols.remove('flow_m3/s')
patched_df = patched_df[cols]

# Save
out_csv = r'../data/tidy/martini_daily_concs_patched.csv'
patched_df.to_csv(out_csv, index=False, encoding='utf-8')

patched_df.head()